In [15]:
from attractions_recc import *
from IPython.display import display
import ipywidgets as w
import pandas as pd
import re

In [16]:
att_df = pd.read_json('etl/attractions.json',orient='records')

## User details and Preferences

In [17]:
uname = w.Text(description="User Name")
place = w.Text(value = 'Province',  description="Destination")
budget = w.IntRangeSlider(min=att_df.price.min(), max=att_df.price.max(),step=10,value=[att_df.price.min(),att_df.price.max()],description="Budget")
v1 = w.VBox([uname,place,budget])
start = w.DatePicker(description='Start Date',disabled=False)
end = w.DatePicker(description='End Date',disabled=False)
v2 = w.VBox([start,end])
out = w.HBox([v1,v2])
display(out)

##### category_df = att_df.groupby('category').size().reset_index().sort_values([0],ascending=False)[:20]
categories = list(category_df.category.values)
cat_rat = dict()
def on_button_clicked(b):
    if b.description in cat_rat:
        return
    print(b.description)
    slider = w.IntSlider(min=0,max=5,step=1,description='Rate')
    display(slider)
    cat_rat[b.description] = slider
    if(len(cat_rat) < 5):
        print("Rate {x} more!\n".format(x=5-len(cat_rat)))
    
but_layout = w.Layout(width='100%', height='100px')
but_items = [w.Button(description=c, layout=but_layout) for c in categories]
on_clk = [item.on_click(on_button_clicked) for item in but_items]
c1 = w.VBox([i for i in but_items[:4]])
c2 = w.VBox([i for i in but_items[4:8]])
c3 = w.VBox([i for i in but_items[8:12]])
c4 = w.VBox([i for i in but_items[12:16]])
c5 = w.VBox([i for i in but_items[16:]])
buttons = w.HBox([c1,c2,c3,c4,c5])
print("Select and rate atleast 5 categories and rate them:")
display(buttons)

In [18]:
user_name = re.sub(' ','_',uname.value.lower())
province = re.sub(' ','_',place.value.lower())
(low,high) = tuple([float(i) for i in budget.value])
begin_date = start.value
end_date = end.value
cat_rating = dict()
for key, value in cat_rat.items():
    cat_rating[key] = float(value.value)

## Model recommendation and filtering

In [19]:
filename, user, rbm_att = get_recc(att_df, cat_rating)
with_url = filter_df(filename, user, low, high, province, att_df)

Reading the data
Extracting num_rows from ratings
Preprocessing the dataset
Similar User: 4068
Model restored


I0000 00:00:1762959106.845097 5250988 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled


,attraction_id,Recommendation Score
0,0,0.018642
1,1,0.003147
2,2,0.000258
3,3,0.000034
4,4,0.000369


In [20]:
%%capture
final = dict()
final['timeofday'] = []
final['image'] = []
final['name'] = []
final['location'] = []
final['price'] = []
final['rating'] = []
final['category'] = []

for i in range(1,(end_date - begin_date).days+2):
    for j in range(2):
        final['timeofday'].append('Morning')
    for j in range(2):
        final['timeofday'].append('Evening')

for i in range(len(final['timeofday'])): 
    if i%4 == 0: 
        final = top_recc(with_url, final)
    else:
        final = find_closest(with_url, final['location'][-1],final['timeofday'][i], final)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

## Final Output

In [21]:
days = (end_date - begin_date).days + 1
display(final_output(days,final))

TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'